In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
import xarray as xr
import numpy as np                
import os, subprocess, glob
from shutil import copy2

In [83]:
new = xr.open_mfdataset('/Users/robert/Codes/rte-rrtmgp/examples/rfmip-clear-sky/r??_template.nc')
ref = xr.open_mfdataset('/Users/robert/Dropbox/Scientific/Projects/RFMIP/RFMIP-IRF/LBLRTM/r??.nc')
new.load()
ref.load()
conds = xr.open_dataset('/Users/robert/Codes/rte-rrtmgp/examples/rfmip-clear-sky/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-0-4_none.nc')
conds.load()             
toa = conds.isel(site=0,expt=0).pres_level.argmin()
sfc = conds.isel(site=0,expt=0).pres_level.argmax()
(toa.values, sfc.values)

(array(0), array(60))

In [82]:
print("Fluxes at present day")
ref_avg = (ref * ref.profile_weight).sum(dim='site')
new_avg = (new * new.profile_weight).sum(dim='site')
print("Mean at toa",             ref_avg.isel(expt=1,level=toa))
print("Error at toa", (new_avg - ref_avg).isel(expt=1,level=toa)) 
print("Mean at sfc",             ref_avg.isel(expt=1,level=sfc))
print("Error at sfc", (new_avg - ref_avg).isel(expt=1,level=sfc)) 


Fluxes at present day
Mean at toa <xarray.Dataset>
Dimensions:         ()
Coordinates:
    lon             float32 27.0
    lat             float32 -28.5
    time            datetime64[ns] 2014-01-05T06:00:00
Data variables:
    plev            float32 0.009999999
    rld             float32 0.0
    profile_weight  float64 0.01053
    rlu             float32 266.04184
    rsd             float32 335.07663
    rsu             float32 47.372665
Error at toa <xarray.Dataset>
Dimensions:         ()
Coordinates:
    lon             float32 27.0
    lat             float32 -28.5
    time            datetime64[ns] 2014-01-05T06:00:00
Data variables:
    plev            float32 0.0
    rld             float32 0.0
    profile_weight  float64 0.0
    rlu             float32 -0.11514282
    rsd             float32 0.11260986
    rsu             float32 0.22745895
Mean at sfc <xarray.Dataset>
Dimensions:         ()
Coordinates:
    lon             float32 27.0
    lat             float32 -28.5
   

In [62]:
# RFMIP results: Error (and forcing) relative to PI
#
# Forcing - Present-day minus pre-industrial (see conds.expt_label)
#
ref_f = ref.isel(expt=0) - ref.isel(expt=1)
new_f = new.isel(expt=0) - new.isel(expt=1)

print("Forcing relative to PI")

# Someone more Pythonic than I would turn this into a data frame. 
ref_f_avg = (ref_f * ref.profile_weight).sum(dim='site')
new_f_avg = (new_f * new.profile_weight).sum(dim='site')

# Up at top of atmosphere
print("Up at TOA")
print("Longwave",  (new_f_avg - ref_f_avg).isel(level=toa).rlu.values, ref_f_avg.isel(level=toa).rlu.values)
print("Shortwave", (new_f_avg - ref_f_avg).isel(level=toa).rsu.values, ref_f_avg.isel(level=toa).rsu.values)

# Down minus up, toa minus surface
ref_f_avg_rln =  ref_f_avg.rld - ref_f_avg.rlu
new_f_avg_rln =  new_f_avg.rld - new_f_avg.rlu
ref_f_avg_rsn =  ref_f_avg.rsd - ref_f_avg.rsu
new_f_avg_rsn =  new_f_avg.rsd - new_f_avg.rsu
print("Net absorption")
print("Longwave", \
      (new_f_avg_rln.isel(level=toa) - new_f_avg_rln.sel(level=sfc)).values - \
      (ref_f_avg_rln.isel(level=toa) - ref_f_avg_rln.sel(level=sfc)).values, \
      (ref_f_avg_rln.isel(level=toa) - ref_f_avg_rln.sel(level=sfc)).values)
print("Shortwave", \
      (new_f_avg_rsn.isel(level=toa) - new_f_avg_rsn.sel(level=sfc)).values - \
      (ref_f_avg_rsn.isel(level=toa) - ref_f_avg_rsn.sel(level=sfc)).values, \
      (ref_f_avg_rsn.isel(level=toa) - ref_f_avg_rsn.sel(level=sfc)).values)

# Down at surface 
print("Down at surface")
print("Longwave",  (new_f_avg - ref_f_avg).isel(level=sfc).rld.values, ref_f_avg.isel(level=sfc).rld.values)
print("Shortwave", (new_f_avg - ref_f_avg).isel(level=sfc).rsd.values, ref_f_avg.isel(level=sfc).rsd.values)

Forcing relative to PI
Up at TOA
Longwave 0.14799976 -2.8446794
Shortwave 0.0070251897 -0.05815421
Net absorption
Longwave -0.054691195 0.8033571
Shortwave -0.05080214 0.52163047
Down at surface
Longwave -0.09520328 2.0829737
Shortwave 0.06421837 -0.53439766


In [84]:
ref.close()
new.close()